## 必要なライブラリのインポート
- numpy,chainer系
- ファイル操作系
- 画像操作

In [1]:
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import iterators, optimizers, training
from chainer.training import extensions
from chainer.cuda import to_cpu

import os
import glob
from PIL import Image

## modelの定義(AlexNetぽい構造の何か)

In [2]:
class AlexNet(chainer.Chain):
#     insize = 100
    def __init__(self):
        super(AlexNet, self).__init__(
            conv1=L.Convolution2D(3, 96, 11, stride=4),
            conv2=L.Convolution2D(96, 256, 5, pad=2),
            conv3=L.Convolution2D(256, 384, 3, pad=1),
            conv4=L.Convolution2D(384, 284, 3, pad=1),
            conv5=L.Convolution2D(284, 256, 3, pad=1),
            fc6=L.Linear(1024, 4096),
            fc7=L.Linear(4096, 1024),
            fc8=L.Linear(1024, 50),
        )

    def __call__(self, x, t=None):
        h = x
        h = F.max_pooling_2d(F.local_response_normalization(F.relu(self.conv1(h))), 3, stride=2)
        h = F.max_pooling_2d(F.local_response_normalization(F.relu(self.conv2(h))), 3, stride=2)
        h = F.relu(self.conv3(h))
        h = F.relu(self.conv4(h))
        h = F.max_pooling_2d(F.relu(self.conv5(h)), 3, stride=2)
        h = F.dropout(F.relu(self.fc6(h)))
        h = F.dropout(F.relu(self.fc7(h)))
        if t is not None:
            loss = F.softmax_cross_entropy(h, t)
            chainer.report({'loss': loss, 'accuracy': F.accuracy(h, t)}, self)
            return loss
        else:
            return to_cpu(F.softmax(self.fc8(h)).data)

## データセットのディレクトリ名の取得
以降、ディレクトリごとにラベルつけていく

In [4]:
dataset_dir = "dataset"
label_names = os.listdir(dataset_dir)

In [5]:
label_names

['94-0',
 '788-0',
 '89-1',
 '786-0',
 '286-0',
 '105-1',
 '6-0',
 '473-0',
 '143-0',
 '257-0',
 '798-0',
 '530-0',
 '658-0',
 '635-0',
 '472-0',
 '645-1',
 '428-0',
 '448-0',
 '3-0',
 '793-0',
 '642-1',
 '260-0',
 '497-0',
 '376-0',
 '450-0',
 '785-0',
 '778-0',
 '598-0',
 '184-0',
 '488-0',
 '681-0',
 '145-0',
 '303-0',
 '787-0',
 '279-0',
 '730-0',
 '38-1',
 '445-0',
 '130-0',
 '797-0',
 '233-0',
 '212-0',
 '485-0',
 '149-0',
 '479-2',
 '637-0',
 '115-0',
 '373-0',
 '248-0',
 '59-0']

In [13]:
_labels = []
_images = []

In [14]:
for i, label in enumerate(label_names):
    for f in glob.glob("%s/%s/*"%(dataset_dir,label)):
        img = np.array(Image.open(f))
        _labels.append(i)
        _images.append(img)

In [15]:
_images[0]

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ..., 
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ..., 
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ..., 
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ..., 
       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ..., 
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ..., 
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ..., 
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [16]:
# 正規化
images = np.array(_images, dtype=np.float32) / 255.0
labels = np.array(_labels, dtype=np.int32)

## chainerで扱えるようにRGBの層を手前に持ってくる

In [17]:
images = images.transpose(0,3,1,2)

In [18]:
labels.shape

(50,)

## 画像と正解ラベルのペアを作成する

In [20]:
x_train = chainer.datasets.TupleDataset(images, labels)

In [21]:
x_train[0]

(array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]], dtype=float32), 0)

## 入力に渡すiteratorの作成

In [23]:
train_iter = iterators.SerialIterator(x_train, batch_size=10, shuffle=True)

## modelの作成

In [24]:
model = AlexNet()

## 最適化手法の選択

In [25]:
optimizer = optimizers.Adam()
optimizer.setup(model)

## undaterとtrainerの作成
device=0で処理が0番GPUに投げられるはず  
訓練結果はresultディレクトリに作成されるはず

In [27]:
updater = training.StandardUpdater(train_iter, optimizer, device=0)

/home/c0115334ef/anaconda3/lib/python3.6/site-packages/chainer/cuda.py:84: UserWarning: cuDNN is not enabled.
Please reinstall CuPy after you install cudnn
(see https://docs-cupy.chainer.org/en/stable/install.html#install-cupy-with-cudnn-and-nccl).
  'cuDNN is not enabled.\n'


In [29]:
trainer = training.Trainer(updater, (100, 'epoch'), out='result')

## Logの出力設定

In [30]:
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'main/loss']))
trainer.extend(extensions.ProgressBar())

In [31]:
trainer.run()

epoch       main/accuracy  main/loss 
1           0              6.95797     
2           0.02           6.93148     
3           0.02           6.93147     
4           0.02           6.93147     
5           0.02           6.93147     
6           0.02           6.93147     
7           0.02           6.93147     
8           0.02           6.93147     
9           0.02           6.93147     
10          0.02           6.93147     
11          0.02           6.93147     
12          0.02           6.93147     
13          0.02           6.93147     
14          0.02           6.93147     
15          0.02           6.93147     
16          0.02           6.93147     
17          0.02           6.93147     
18          0.02           6.93147     
19          0.02           6.93147     
20          0.02           6.93147     
     total [##########........................................] 20.00%
this epoch [..................................................]  0.00%
       100 iter, 20 

## とりあえず実行できる形にはなったけど何も学んでない感じがある